<a href="https://colab.research.google.com/github/imabari/covid19-data/blob/master/vaccine/%E5%89%AF%E5%8F%8D%E5%BF%9C%E7%96%91%E3%81%84%E5%A0%B1%E5%91%8A%E9%9B%86%E8%A8%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://www.mhlw.go.jp/content/10906000/000790066.pdf -O data.pdf

--2021-06-09 11:42:48--  https://www.mhlw.go.jp/content/10906000/000790066.pdf
Resolving www.mhlw.go.jp (www.mhlw.go.jp)... 69.192.197.216, 2600:1405:800:1a3::3194, 2600:1405:800:194::3194
Connecting to www.mhlw.go.jp (www.mhlw.go.jp)|69.192.197.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5600642 (5.3M) [application/pdf]
Saving to: ‘data.pdf’

data.pdf            100%[===================>]   5.34M  --.-KB/s    in 0.06s   

2021-06-09 11:42:49 (85.8 MB/s) - ‘data.pdf’ saved [5600642/5600642]



In [2]:
!apt install python3-tk ghostscript
!pip install camelot-py[cv]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-tk is already the newest version (3.6.9-1~18.04).
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono gsfonts libcupsfilters1 libcupsimage2
  libgs9 libgs9-common libijs-0.35 libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto ghostscript-x poppler-utils fonts-japanese-mincho
  | fonts-ipafont-mincho fonts-japanese-gothic | fonts-ipafont-gothic
  fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts libcupsfilters1
  libcupsimage2 libgs9 libgs9-common libijs-0.35 libjbig2dec0 poppler-data
0 upgraded, 11 newly installed, 0 to remove and 39 not upgraded.
Need to get 14.1 MB of archives.
After this operation, 49.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 fonts-droid-fallback all 1:6.0.

In [3]:
import camelot
import pandas as pd

# PDF変換

In [4]:
tables = camelot.read_pdf("data.pdf", pages="2-end", split_text=True)

df = pd.concat([table.df for table in tables])

df.to_csv("data.csv", encoding="utf_8_sig", index=False, header=False)

# ダウンロード

In [5]:
from google.colab import files

In [6]:
files.download("data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 前処理

In [7]:
import re
import pandas as pd

In [8]:
#!wget https://raw.githubusercontent.com/imabari/covid19-data/master/vaccine/data.csv -O data.csv

In [9]:
df0 = pd.read_csv("data.csv", dtype="object").fillna("")

In [10]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10659 entries, 0 to 10658
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   No             10659 non-null  object
 1   年齢             10659 non-null  object
 2   性別             10659 non-null  object
 3   接種日            10659 non-null  object
 4   発生日            10659 non-null  object
 5   接種から
発生までの
日数  10659 non-null  object
 6   ワクチン名          10659 non-null  object
 7   製造販売業者         10659 non-null  object
 8   ロット番
号         10659 non-null  object
 9   症状名（PT名）       10659 non-null  object
 10  因果関係
（報告医評価）   10659 non-null  object
 11  重篤度
（報告医評価）    10659 non-null  object
 12  転帰日            10659 non-null  object
 13  転帰内容           10659 non-null  object
dtypes: object(14)
memory usage: 1.1+ MB


In [11]:
df0.columns = df0.columns.str.replace("\n", "")

In [12]:
df0["No"] = df0["No"].str.replace("※[１-９]", "").str.strip().astype(int)

In [13]:
df0["年齢"] = df0["年齢"].str.rstrip("歳")
df0["年齢"] = df0["年齢"].mask(df0["年齢"] == "").astype(float).astype("Int64")

In [14]:
df0["性別"] = df0["性別"].replace({"": "不明"})

In [15]:
df0["症状名（PT名）"] = df0["症状名（PT名）"].str.normalize("NFKC").str.strip().str.replace(" ", "")

# 整然データ

かのえ／Pattie／柿人 さんのコードを参考にさせていただきました

https://twitter.com/kanoekakihito/status/1397222333505486849

https://github.com/kanoekakihito/tidy_mhlw_data/blob/main/20210523_%E6%96%B0%E5%9E%8B%E3%82%B3%E3%83%AD%E3%83%8A%E3%83%AF%E3%82%AF%E3%83%81%E3%83%B3%E5%89%AF%E5%8F%8D%E5%BF%9CCSV%E6%95%B4%E5%BD%A2.py

In [16]:
df1 = df0.copy()

In [17]:
df1

,No,年齢,性別,接種日,発生日,接種から発生までの日数,ワクチン名,製造販売業者,ロット番号,症状名（PT名）,因果関係（報告医評価）,重篤度（報告医評価）,転帰日,転帰内容
0,1,55,女性,2021/03/19,2021/03/19\r\n2021/03/19,0,コミナティ筋注,ファイザー,EP9605,口腔咽頭不快感(口腔咽頭不快感)\r\n皮疹・発疹・紅斑(紅斑),,重くない,未記入\r\n未記入,不明\r\n軽快
1,2,24,女性,2021/04/01,2021/04/01\r\n2021/04/01,0,コミナティ筋注,ファイザー,ER2659,悪心・嘔吐(悪心)\r\n口腔咽頭不快感(口腔咽頭不快感),,重くない,未記入\r\n未記入,軽快\r\n軽快
2,3,77,男性,2021/04/19,2021/04/23,4,コミナティ筋注,ファイザー,ET9096,脳出血(脳幹出血),,重い,未記入,死亡
3,4,66,男性,2021/05/08,2021/05/08\r\n2021/05/08,0,コミナティ筋注,ファイザー,ER7449,発熱(発熱)\r\n接種部位疼痛(ワクチン接種部位疼痛),,重くない,2021/05/10\r\n2021/05/10,回復\r\n回復
4,5,61,女性,2021/04/20,2021/04/20\r\n2021/04/20,0,コミナティ筋注,ファイザー,不明,頭痛(頭痛)\r\n血圧上昇(血圧上昇),,重い,2021/04/25\r\n2021/04/22,回復\r\n回復
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10654,10672,52,女性,2021/05/18,2021/05/19\r\n2021/05/19,1,コミナティ筋注,ファイザー,EY5420,発熱(発熱)\r\n接種部位疼痛(ワクチン接種部位疼痛),関連あり,重くない,2021/05/20\r\n2021/05/20,回復\r\n回復
10655,10673,51,女性,2021/05/18,2021/05/18\r\n2021/05/18,0,コミナティ筋注,ファイザー,EY2173,口腔咽頭不快感(口腔咽頭不快感)\r\n熱感(熱感),関連あり,重くない,未記入\r\n未記入,軽快\r\n軽快
10656,10674,70,女性,2021/05/18,2021/05/18,0,コミナティ筋注,ファイザー,EY2173,頻脈(頻脈),関連なし,重くない,2021/05/18,回復
10657,10675,58,女性,2021/05/28,2021/05/28,0,コミナティ筋注,ファイザー,EY2173,動悸(動悸),関連あり,重くない,未記入,軽快


In [18]:
len(df1.loc[df1["転帰内容"].str.contains("死亡"), "No"])

123

In [19]:
df1["症状名（PT名）"] = df1["症状名（PT名）"].str.replace("\r", "").apply(lambda s: [i.replace("\n", "").strip() for i in re.split("(?<=\)\n)", s)])
df1["発生日"] = df1["発生日"].str.split()
df1["転帰日"] = df1["転帰日"].str.split()
df1["転帰内容"] = df1["転帰内容"].str.split()

In [20]:
df1["症状名（PT名）"]

0        [口腔咽頭不快感(口腔咽頭不快感), 皮疹・発疹・紅斑(紅斑)]
1           [悪心・嘔吐(悪心), 口腔咽頭不快感(口腔咽頭不快感)]
2                             [脳出血(脳幹出血)]
3            [発熱(発熱), 接種部位疼痛(ワクチン接種部位疼痛)]
4                    [頭痛(頭痛), 血圧上昇(血圧上昇)]
                       ...               
10654        [発熱(発熱), 接種部位疼痛(ワクチン接種部位疼痛)]
10655          [口腔咽頭不快感(口腔咽頭不快感), 熱感(熱感)]
10656                            [頻脈(頻脈)]
10657                            [動悸(動悸)]
10658                        [そう痒症(そう痒症)]
Name: 症状名（PT名）, Length: 10659, dtype: object

In [21]:
dfs = []

for i, row in df1.iterrows():
    dfs.append(row.apply(pd.Series).T.fillna(method="ffill"))

In [22]:
df2 = pd.concat(dfs).reset_index(drop=True)

In [23]:
df2["症状名（PT名）"]

0        口腔咽頭不快感(口腔咽頭不快感)
1            皮疹・発疹・紅斑(紅斑)
2               悪心・嘔吐(悪心)
3        口腔咽頭不快感(口腔咽頭不快感)
4               脳出血(脳幹出血)
               ...       
24380    口腔咽頭不快感(口腔咽頭不快感)
24381              熱感(熱感)
24382              頻脈(頻脈)
24383              動悸(動悸)
24384          そう痒症(そう痒症)
Name: 症状名（PT名）, Length: 24385, dtype: object

In [24]:
df3 = df2["症状名（PT名）"].str.extract("(.+)\((.+?)\)").rename(columns={0:"症状名", 1:"PT名"})
df3["PT名"] = df3["PT名"].str.split("|")

In [25]:
df4 = df2.join(df3).explode("PT名")

In [26]:
df4["症状名"] = df4["症状名"].str.strip()
df4["PT名"] = df4["PT名"].str.strip()

In [27]:
df4["No"] = df2["No"].astype(int)
df4["年齢"] = df2["年齢"].astype("Int64")

In [28]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25032 entries, 0 to 24384
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   No           25032 non-null  int64 
 1   年齢           25031 non-null  Int64 
 2   性別           25032 non-null  object
 3   接種日          25032 non-null  object
 4   発生日          25032 non-null  object
 5   接種から発生までの日数  25032 non-null  object
 6   ワクチン名        25032 non-null  object
 7   製造販売業者       25032 non-null  object
 8   ロット番号        25032 non-null  object
 9   症状名（PT名）     25032 non-null  object
 10  因果関係（報告医評価）  25032 non-null  object
 11  重篤度（報告医評価）   25032 non-null  object
 12  転帰日          25032 non-null  object
 13  転帰内容         25032 non-null  object
 14  症状名          25032 non-null  object
 15  PT名          25032 non-null  object
dtypes: Int64(1), int64(1), object(14)
memory usage: 3.3+ MB


In [29]:
# 集計用に修正

df4["症状名"] = df4["症状名"].str.replace("̲", "_")

df4["症状名"] = df4["症状名"].mask((df4["症状名"] == "失神寸前の状態") & (df4["PT名"] == "失神寸前の状態"), "血管迷走神経反射")
df4["症状名"] = df4["症状名"].mask((df4["症状名"] == "失神") & (df4["PT名"] == "失神"), "血管迷走神経反射_失神を伴う")

In [30]:
df4

,No,年齢,性別,接種日,発生日,接種から発生までの日数,ワクチン名,製造販売業者,ロット番号,症状名（PT名）,因果関係（報告医評価）,重篤度（報告医評価）,転帰日,転帰内容,症状名,PT名
0,1,55,女性,2021/03/19,2021/03/19,0,コミナティ筋注,ファイザー,EP9605,口腔咽頭不快感(口腔咽頭不快感),,重くない,未記入,不明,口腔咽頭不快感,口腔咽頭不快感
1,1,55,女性,2021/03/19,2021/03/19,0,コミナティ筋注,ファイザー,EP9605,皮疹・発疹・紅斑(紅斑),,重くない,未記入,軽快,皮疹・発疹・紅斑,紅斑
2,2,24,女性,2021/04/01,2021/04/01,0,コミナティ筋注,ファイザー,ER2659,悪心・嘔吐(悪心),,重くない,未記入,軽快,悪心・嘔吐,悪心
3,2,24,女性,2021/04/01,2021/04/01,0,コミナティ筋注,ファイザー,ER2659,口腔咽頭不快感(口腔咽頭不快感),,重くない,未記入,軽快,口腔咽頭不快感,口腔咽頭不快感
4,3,77,男性,2021/04/19,2021/04/23,4,コミナティ筋注,ファイザー,ET9096,脳出血(脳幹出血),,重い,未記入,死亡,脳出血,脳幹出血
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24380,10673,51,女性,2021/05/18,2021/05/18,0,コミナティ筋注,ファイザー,EY2173,口腔咽頭不快感(口腔咽頭不快感),関連あり,重くない,未記入,軽快,口腔咽頭不快感,口腔咽頭不快感
24381,10673,51,女性,2021/05/18,2021/05/18,0,コミナティ筋注,ファイザー,EY2173,熱感(熱感),関連あり,重くない,未記入,軽快,熱感,熱感
24382,10674,70,女性,2021/05/18,2021/05/18,0,コミナティ筋注,ファイザー,EY2173,頻脈(頻脈),関連なし,重くない,2021/05/18,回復,頻脈,頻脈
24383,10675,58,女性,2021/05/28,2021/05/28,0,コミナティ筋注,ファイザー,EY2173,動悸(動悸),関連あり,重くない,未記入,軽快,動悸,動悸


In [31]:
df4.to_csv("output.csv", index=False, encoding="utf_8_sig")

# ダウンロード

In [32]:
files.download("output.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 症状別報告件数

type.csvは
症状別報告件数集計確認.ipynb
で利用する

In [33]:
df4["症状"] = df4["症状名"] + "(" + df4["PT名"] + ")"

In [34]:
df4["症状"].value_counts().to_csv("type.csv", header=False, encoding="utf_8_sig")

In [35]:
files.download("type.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 集計確認

In [36]:
df1 = df0.copy()
df1.set_index("No", inplace=True)

## アナフィラキシー

In [37]:
df1["症状名（PT名）"].str.contains("アナフィラキシー").sum()

1263

## 性別

In [38]:
# 性別 副反応疑い報告数
df1["性別"].value_counts()

女性    8729
男性    1911
不明      19
Name: 性別, dtype: int64

In [39]:
# 性別 重篤報告数
df1.loc[df1["重篤度（報告医評価）"] == "重い", "性別"].value_counts()

女性    1008
男性     249
不明       4
Name: 性別, dtype: int64

In [40]:
# 性別 死亡報告数
df1.loc[df1["転帰内容"].str.contains("死亡"), "性別"].value_counts()

女性    64
男性    59
Name: 性別, dtype: int64

## 年齢別

In [41]:
labels = [
    "0～9歳",
    "10～19歳",
    "20～29歳",
    "30～39歳",
    "40～49歳",
    "50～59歳",
    "60～69歳",
    "70～79歳",
    "80歳以上",
]

In [42]:
df1["年代"] = pd.cut(
    df1["年齢"],
    [0, 10, 20, 30, 40, 50, 60, 70, 80, 120],
    include_lowest=True,
    right=False,
    labels=labels,
).astype(str)

In [43]:
df1["年代"] = df1["年代"].mask(df1["年齢"].isnull(), "不明")

In [44]:
# 年齢別副反応疑い報告数
df1["年代"].value_counts().reindex(labels + ["不明"]).fillna(0).astype(int)

0～9歳         0
10～19歳      53
20～29歳    2015
30～39歳    2399
40～49歳    2993
50～59歳    1921
60～69歳     672
70～79歳     236
80歳以上      369
不明           1
Name: 年代, dtype: int64

In [45]:
# 年齢別重篤報告数
df1.loc[df1["重篤度（報告医評価）"] == "重い", "年代"].value_counts().reindex(labels + ["不明"]).fillna(0).astype(int)

0～9歳        0
10～19歳      8
20～29歳    199
30～39歳    260
40～49歳    321
50～59歳    193
60～69歳     72
70～79歳     65
80歳以上     142
不明          1
Name: 年代, dtype: int64

In [46]:
# 年齢別死亡報告数
df1.loc[df1["転帰内容"].str.contains("死亡"), "年代"].value_counts().reindex(labels + ["不明"]).fillna(0).astype(int)

0～9歳       0
10～19歳     0
20～29歳     3
30～39歳     1
40～49歳     6
50～59歳     4
60～69歳    10
70～79歳    23
80歳以上     76
不明         0
Name: 年代, dtype: int64